In [ ]:
import json
import os
import numpy as np
import time
import torch
import model_training.config as config
import model_training.utils as utils
import inference.single_image_inference as inference
import preprocessing.preprocess_data as preprocessing
from datetime import datetime
from mpl_toolkits.mplot3d import Axes3D
from model_training.model import YOLOv3


# Run Inference Continuously

In [7]:


current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
run_id = f"{config.INFERENCE_RUN_TITLE}_{current_datetime}"

# Create a folder for the run_id inside 'inference/inference_results/'
results_folder = os.path.join(config.INFERENCE_RESULTS_FOLDER, run_id)
processed_pcd_folder = config.INFERENCE_PROCESSED_PCD_FOLDER
results_file_path = os.path.join(results_folder, f"results_{run_id}.json")
os.makedirs(results_folder, exist_ok=True)
os.makedirs(processed_pcd_folder, exist_ok=True)


model = YOLOv3(num_classes=config.NUM_CLASSES).to(config.DEVICE)
utils.load_checkpoint(
            config.INFERENCE_CHECKPOINT_FILE, model, None, None
        )
# Inference loop
while True:
    # Monitor the folder for new point cloud files
    for file_name in os.listdir(config.INFERENCE_PCD_FOLDER):
        print(file_name)
        if file_name.endswith(".pcd"):
            # Transform point cloud to BEV image
            bev_image = preprocessing.transform_to_bev(file_name)

            # Extract x, y coordinates from YOLO v3 output
            bboxes_pred = inference.inference_single_image(model, bev_image)

            # Print results to console
            print(f"File: {file_name}, bboxes: {bboxes_pred}")

            result_dict = {}
            for idx in range(len(bboxes_pred)):
                bbox_info = {
                    "confidence": bboxes_pred[idx][2],
                    "x": bboxes_pred[idx][3],
                    "y": bboxes_pred[idx][4],
                    "w": bboxes_pred[idx][5],
                    "h": bboxes_pred[idx][6]
                }
                result_dict[idx] = bbox_info

            # Create a dictionary to store all results
            results = {}

            # Check if the results file already exists
            if os.path.exists(results_file_path):
                # Read existing results from the file
                with open(results_file_path, 'r') as json_file:
                    results = json.load(json_file)

            # Add new results to the dictionary
            results[file_name] = result_dict

            # Write all results back to the file
            with open(results_file_path, "w") as json_file:
                json.dump(results, json_file, indent=2)

            # Optional: Move processed file to another folder to avoid re-processing
            os.rename(os.path.join(config.INFERENCE_PCD_FOLDER, file_name),
                      os.path.join(processed_pcd_folder, file_name))

    # Sleep for a while before checking for new files again
    time.sleep(0.01)

=> Loading checkpoint: inference/model/inference_model_checkpoint.pth.tar
000001.pcd
Image saved at inference/temp/bev_images\000001.png


100%|██████████| 1/1 [00:09<00:00,  9.99s/it]

Starting NMS
Num boxes above confidence threshold: 3
Highest Box confidence scores: [0.8222513198852539, 0.8206561803817749, 0.813751757144928]


100%|██████████| 1/1 [00:10<00:00, 10.38s/it]


File: 000001.pcd, bboxes: [[0, 0.0, 0.8222513198852539, 0.6191576719284058, 0.3400348424911499, 0.027623251080513, 0.027230486273765564], [0, 0.0, 0.8206561803817749, 0.42888298630714417, 0.37422239780426025, 0.027232525870203972, 0.025846345350146294]]
000002.pcd
Image saved at inference/temp/bev_images\000002.png


...

100%|██████████| 1/1 [00:09<00:00,  9.98s/it]

Starting NMS
Num boxes above confidence threshold: 2
Highest Box confidence scores: [0.843324601650238, 0.8375163078308105]


100%|██████████| 1/1 [00:10<00:00, 10.37s/it]


File: 000010.pcd, bboxes: [[0, 0.0, 0.843324601650238, 0.6198647022247314, 0.33993402123451233, 0.027367567643523216, 0.02721094712615013]]


KeyboardInterrupt: 

## Plot results ontop of BEV images